<a href="https://colab.research.google.com/github/zmohaveri/IVT-ActiveLearning/blob/main/IVT_dispendix_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#imports

In [23]:
import numpy as np
from collections import OrderedDict 
import pandas as pd
from shutil import copyfile
from string import ascii_uppercase as alphabets
import datetime as tm
from math import ceil

#Random samples

##Making random points

In [2]:
def random_points_in_range(n,ranges):
  points = np.empty((n,len(ranges)))
  for i,element in enumerate(ranges):
    start=min(element[1],element[0])
    interval=abs(element[1]-element[0])
    randoms = ( np.random.rand(n)*interval ) + start
    points[:,i] = randoms.T
  return points

##adding pipetting action for each point to a CSV

In [47]:
def add_line_to_file(line,file_path,mode='a'):
  with open(file_path,mode) as file:
    file.write('\n'+line)

def concentration_to_volume(sample_volume,concentration,stock_concentration):
  #c1.v1=c2.v2 => v1=c2.v2/c1
  return concentration*sample_volume/stock_concentration

def concentration_points_to_volumes(points,sample_volume,scanning_elements):
  max_volume=0
  volume_points=np.empty(points.shape)
  sums=np.zeros(points.shape[1:])
  for j,point in enumerate(points):
    volume_point=np.empty(points.shape[1:])
    for i,element in enumerate(scanning_elements):
      volume= concentration_to_volume(sample_volume,point[i],scanning_elements[element][1])
      sums[i] += volume
      source=scanning_elements[element][0]
      volume_point[i] = volume
    max_volume= max( np.sum(volume_point) , max_volume)
    volume_points[j]=volume_point
  return volume_points,max_volume,sums

def add_dispense_to_file(volume_point,destination,fill_up_to,water_position,scanning_elements,file_path):
  for i,element in enumerate(scanning_elements):
    #volume = concentration_to_volume(sample_volume,point[i],scanning_elements[element][1])
    source = scanning_elements[element][0]
    add_line_to_file(f'{source},{destination},{volume_point[i]},{element}',file_path)
  water_volume = fill_up_to - np.sum(volume_point)
  add_line_to_file(f'{water_position},{destination},{water_volume},Water',file_path)
  return water_volume

def collection_to_csv_keys(collection):
  csv=''
  for element in collection:
    csv += f'{element},'
  return csv

def points_to_protocol(points,destination_wells,sample_volume,scanning_elements,water_position,protocol_path,record_path):
  add_line_to_file(f'destinations,{collection_to_csv_keys(scanning_elements)}',record_path,mode='a')
  volume_points,max_volume,sums=concentration_points_to_volumes(points,sample_volume,scanning_elements)
  max_volume=ceil(max_volume)
  for i,volum_point in enumerate(volume_points):
    add_dispense_to_file(volum_point,destination_wells[i],max_volume,water_position,scanning_elements,protocol_path)
    add_line_to_file(f'{destination_wells[i]},{collection_to_csv_keys(points[i])}',record_path)
  return max_volume,sums

def make_destination_wells(starting_number,finishing_number,starting_letter,finishing_letter):
  destination_wells=[]
  for letter in alphabets[alphabets.find(starting_letter):alphabets.find(finishing_letter)+1]:
    for number in range(starting_number,finishing_number+1):
      destination_wells.append(f'{letter}{number}')
  return destination_wells


In [49]:
sample_volume=20

label=f'2020-11-10-{tm.datetime.now()}-'
empty_path='/content/drive/My Drive/Thesis/IVT/dispendix/CSV_empty_header.csv'
protocol_path=f'/content/drive/My Drive/Thesis/IVT/dispendix/{label}protocol.csv'
record_path=f'/content/drive/My Drive/Thesis/IVT/dispendix/{label}concentrations.csv'
copyfile(empty_path,protocol_path)

scanning_elements = OrderedDict({'spermidine':('B1',1000),'template':('C1',5000),'mgcl2':('D1',2000),'rntp':('E1',25)})
water_position='A1'
random_samples = random_points_in_range(96,[(0,10),(0,300),(0,150),(0,5)])
destination_wells1 = make_destination_wells(19,24,'A','P')
destination_wells2 = make_destination_wells(13,18,'A','P')#rep2
points_to_protocol(random_samples,destination_wells1,sample_volume,scanning_elements,water_position,protocol_path,record_path)
max_volume,sums=points_to_protocol(random_samples,destination_wells2,sample_volume,scanning_elements,water_position,protocol_path,record_path)
print(max_volume)
print(sums*2)

6
[ 19.77625379 123.16260055 132.91132726 403.20234742]


In [29]:
print(random_samples)

[[9.11427139e+00 9.27115350e+01 5.32010420e+01 2.51054522e+00]
 [4.74400563e+00 1.40836565e+02 1.36926803e+02 1.71623440e+00]
 [5.30417894e+00 1.41106780e+02 4.93742986e+01 1.21574497e+00]
 [1.28379448e+00 2.69975753e+02 2.45298956e+01 1.20943157e+00]
 [4.46008484e+00 2.65291665e+02 1.16193795e+02 1.27485888e+00]
 [1.03725041e+00 4.08860763e+01 1.32170007e+02 4.06042868e+00]
 [9.98375529e+00 1.28472396e+01 8.33650979e+01 1.77447684e+00]
 [2.87433221e+00 2.76173767e+02 1.36516110e+02 2.70014999e+00]
 [4.01876319e+00 3.02412471e+01 3.57586209e+01 4.85349655e+00]
 [8.47914269e+00 1.38952636e+02 1.25210832e+02 6.30951224e-01]
 [1.09316988e+00 2.88675265e+02 1.37637134e+01 2.13198975e+00]
 [2.54706141e+00 2.86701814e+02 2.69910630e+01 3.30253919e+00]
 [6.08782564e+00 2.20487054e+02 7.48305200e+01 3.50744771e+00]
 [8.24653088e+00 2.76612135e+01 7.93110697e+01 2.43462727e+00]
 [9.80316720e+00 9.14650749e+01 1.33289978e+02 1.20699103e+00]
 [2.15074205e+00 9.85061003e+01 1.08082539e+02 4.749078

In [12]:
print(np.sum(random_samples,axis=1).shape)
print(random_samples[0])
print(random_samples.shape[1:])
copy=np.empty(random_samples.shape)
for point in random_samples:
  next(copy)=point
print(copy.shape)

SyntaxError: ignored

#distribution

In [5]:
def plot_3d_distribution (xs,ys,zs,name,xlabel='x',ylabel='y',zlabel='z'):
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  ax.scatter(xs, ys, zs, zdir='z')
  ax.set_xlabel(xlabel)
  ax.set_ylabel(ylabel)
  ax.set_zlabel(zlabel)
  plt.show()
  plt.savefig(saving_path(f'distribution-{name}','jpg'))
  plt.clf()

def plot_2d_distribution (x,y,name,xlabel='x',ylabel='y'):
  #fig, ax = plt.subplots(dimensions[0],dimensions[1],figsize=100)
  fig, ax = plt.subplots()
  plt.scatter(x,y,alpha=0.2)
  ax.set_xlabel(xlabel)
  ax.set_ylabel(ylabel)
  plt.show()
  plt.savefig(saving_path(f'distribution-{name}','jpg'))
  plt.clf()

In [ ]:
comb=combinations(scanning_elements,3)
for names in comb:
  print(names)
  xlabel,ylabel,zlabel=names
  xs=scanning_concentrations[xlabel]
  ys=scanning_concentrations[ylabel]
  zs=scanning_concentrations[zlabel]
  print(xs.shape,ys.shape,zs.shape)
  plot_3d_distribution(xs,ys,zs,str(names),xlabel,ylabel,zlabel)

In [ ]:
comb=combinations(scanning_profile,2)
for names in comb:
  print(names)
  xlabel,ylabel=names
  xs=scanning_concentrations[xlabel]
  ys=scanning_concentrations[ylabel]
  plot_2d_distribution(xs,ys,str(names),xlabel,ylabel)